---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [27]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [68]:
zillow = pd.read_csv("City_Zhvi_AllHomes.csv")

GDP = pd.read_excel("gdplev.xls", skiprows= 7, parse_cols = "A:C, E:G")
GDP.columns = ["year", "year GDP", "2009 GDP", "year Qrt", "Qrt GDP", "2009 Qrt GDP"]
my_gdp = GDP[GDP['year Qrt'] >= "2000"].reset_index(drop=True).drop(['year', "year GDP", "2009 GDP","Qrt GDP"], 1)
GDP.head()

,year,year GDP,2009 GDP,year Qrt,Qrt GDP,2009 Qrt GDP
0,1929.0,104.6,1056.6,1947q1,243.1,1934.5
1,1930.0,92.2,966.7,1947q2,246.3,1932.3
2,1931.0,77.4,904.8,1947q3,250.1,1930.3
3,1932.0,59.5,788.2,1947q4,260.3,1960.7
4,1933.0,57.2,778.3,1948q1,266.2,1989.5


In [61]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    
    state_set = {'Ohio' ,'Kentucky', 'American Samoa', 'Nevada', 'Wyoming', 'National', 'Alabama', 'Maryland', 'Alaska', 'Utah',  'Oregon', 'Montana',  'Illinois',  'Tennessee', 'District of Columbia',  'Vermont','Idaho',  'Arkansas',  'Maine',  'Washington', 'Hawaii', 'Wisconsin', 'Michigan', 'Indiana', 'New Jersey', 'Arizona', 'Guam',  'Mississippi', 'Puerto Rico', 'North Carolina', 'Texas',  'South Dakota', 'Northern Mariana Islands', 'Iowa',  'Missouri',  'Connecticut',  'West Virginia', 'South Carolina', 'Louisiana',  'Kansas', 'New York',  'Nebraska',  'Oklahoma',  'Florida',  'California', 'Colorado',  'Pennsylvania', 'Delaware',  'New Mexico',  'Rhode Island',  'Minnesota',  'Virgin Islands',  'New Hampshire',  'Massachusetts',  'Georgia', 'North Dakota',  'Virginia'}
    uni_town_list = []
    with open('university_towns.txt') as uni_town:
        for line in uni_town:
            row = line.split("[")[0].split("(")[0]
    
            if row in state_set:
                state_name = row
            else:
                uni_town_list.append([state_name.strip() , row.strip()])
            
            
    unitowns = pd.DataFrame(uni_town_list, columns=["State", "RegionName"])
    return unitowns

In [65]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    
    for index, row in GDP.iterrows():
        if (index < len(GDP) - 2)  and float(row["year Qrt"][:-2]) > 2000:
            if (GDP["Qrt GDP"].iloc[index+2] < GDP["Qrt GDP"].iloc[index+1]) and (GDP["Qrt GDP"].iloc[index+1] < GDP["Qrt GDP"].iloc[index]):
                return GDP["year Qrt"].iloc[index]
        
    return ""
get_recession_start()

'2008q3'

In [66]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    
    recession_start = get_recession_start()
    
    for index, row in GDP.iterrows():
        if (index > 1 and index < len(GDP) - 2)  and float(row["year Qrt"][:-2]) > 2000:
            if (GDP["Qrt GDP"].iloc[index+2] > GDP["2009 Qrt GDP"].iloc[index+1]) and (GDP["Qrt GDP"].iloc[index+1] > GDP["Qrt GDP"].iloc[index]) and (GDP["Qrt GDP"].iloc[index - 1] >= GDP["Qrt GDP"].iloc[index]):        
                if GDP["year Qrt"].iloc[index+2] > recession_start:
                    return GDP["year Qrt"].iloc[index+2]
       
    return "ANSWER"
get_recession_end()

'2009q4'

In [67]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    
    bottom = GDP["Qrt GDP"][GDP["year Qrt"] < recession_end][GDP["year Qrt"] > recession_start].min()
    
    
    
    bottom_year = GDP.loc[GDP["Qrt GDP"] == bottom, "year Qrt"].item()
    
    return bottom_year
get_recession_bottom()

'2009q2'

In [8]:
zillow = pd.read_csv("City_Zhvi_AllHomes.csv")

In [9]:
print(len(zillow))
zillow.head()

10730


,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [31]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    columns = list(zillow.columns)
    columns_19s = []
    columns_no_need = ["RegionID", "Metro", "CountyName", "SizeRank"]
    
    for idx in range(5, len(columns)):    # finding the 19s columns
        if columns[idx][:4] < "2000":
            columns_19s.append(columns[idx])
           
    for col in columns_19s:            # removing 19s values from the columns list 
        columns.remove(col)
    
    for col in columns_no_need:       # removing "RegionID", "Metro", "CountyName", "SizeRank" from the columns list
        columns.remove(col)
        
    new_zillow = zillow[columns].copy()
    quarter_zillow = new_zillow[["RegionName", "State"]].copy()
    quarter_zillow["State"].replace(states, inplace=True)
    
    #quarter_zillow = quarter_zillow.replace({'State':states})  # this is an alternative way
    
    
    for val in range(2000,2016):
        
        quarter_zillow[str(val) + "q1"] = new_zillow[[str(val)+"-01", str(val)+"-02", str(val)+"-03"]].mean(axis=1)
        quarter_zillow[str(val) + "q2"] = new_zillow[[str(val)+"-04", str(val)+"-05", str(val)+"-06"]].mean(axis=1)
        quarter_zillow[str(val) + "q3"] = new_zillow[[str(val)+"-07", str(val)+"-08", str(val)+"-09"]].mean(axis=1)
        quarter_zillow[str(val) + "q4"] = new_zillow[[str(val)+"-10", str(val)+"-11", str(val)+"-12"]].mean(axis=1)
    
    year = 2016
    
    quarter_zillow[str(year) + "q1"] = new_zillow[[str(year)+"-01", str(year)+"-02", str(year)+"-03"]].mean(axis=1)
    quarter_zillow[str(year) + "q2"] = new_zillow[[str(year)+"-04", str(year)+"-05", str(year)+"-06"]].mean(axis=1)
    quarter_zillow[str(year) + "q3"] = new_zillow[[str(year)+"-07", str(year)+"-08"]].mean(axis=1)
    
    quarter_zillow = quarter_zillow.set_index(['State','RegionName'])
    
    return quarter_zillow
convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [155]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    unitown = get_list_of_university_towns()
    quarter_zillow = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    bottom_year = get_recession_bottom()
    
    rec_bot_columns = []
    #columns = list(quarter_zillow.columns) 
    
    for col in list(quarter_zillow.columns):
        if col >= recession_start and col <= bottom_year:
            rec_bot_columns.append(col)
    
    rec_bot_data = quarter_zillow[rec_bot_columns].copy()
    
    unitown_list = []
    for idx, row in unitown.iterrows():
        unitown_list.append((row["State"], row["RegionName"]))
    
    nonunitown_list = []
    
    for idx, row in rec_bot_data.iterrows():

        if idx not in unitown_list:
            nonunitown_list.append(idx)

            
    
    unitown_price = rec_bot_data.loc[unitown_list]   
    unitown_price.index.set_names(["State", "RegionName"], inplace=True)
    unitown_price["diff"] = unitown_price[bottom_year] - unitown_price[recession_start]
        
    
    non_unitown_price = rec_bot_data.loc[nonunitown_list]
    non_unitown_price["diff"] = non_unitown_price[bottom_year] - non_unitown_price[recession_start]
    
    t,p = ttest_ind(unitown_price['diff'].dropna(),non_unitown_price['diff'].dropna())
    
    different = True if p < 0.01 else False

    better = "non-university town" if unitown_price['diff'].mean() < non_unitown_price['diff'].mean() else "university town"
    
    return different, p, better
run_ttest()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, 0.0075442926210657919, 'university town')